In [242]:
from webscraper.scrap_modules import my_NLP as myNLP 
import pickle
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Test Code - NLP

load cleaned dataframe

In [243]:
with open("webscraper/cleaned_data/cleaned_dataframe.pkl", "rb") as file:
    df = pickle.load(file)
    

In [244]:
df.head(2)

,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url,keyword,language
0,3685669741,Software Engineer Fullstack (m/w/d),Atruvia AG,"Aschheim, Bavaria, Germany",1 day ago,VollzeitWir sind der Digitalisierungspartner d...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Scientist,de
1,3719620327,Data Analyst - Business Intelligence,Almedia,"Berlin, Berlin, Germany",5 days ago,Almedia helps companies grow by promoting thei...,Mid-Senior level,Full-time,Analyst,"Technology, Information and Internet",2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en


#### Keyword Extraction 
https://www.analyticsvidhya.com/blog/2022/03/keyword-extraction-methods-from-documents-in-nlp/#:~:text=Textrank%20is%20a%20Python%20tool,compatible%20with%20the%20Spacy%20pipeline

In [245]:
import spacy 
import pytextrank 
import pandas as pd
import re

# !spacy download en_core_web_sm
# !spacy download de_core_news_sm

### first cleaning

In [246]:
nlp_df = pd.DataFrame(df) # create a new data frame

In [247]:
nlp_df.head(10)

,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url,keyword,language
0,3685669741,Software Engineer Fullstack (m/w/d),Atruvia AG,"Aschheim, Bavaria, Germany",1 day ago,VollzeitWir sind der Digitalisierungspartner d...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Scientist,de
1,3719620327,Data Analyst - Business Intelligence,Almedia,"Berlin, Berlin, Germany",5 days ago,Almedia helps companies grow by promoting thei...,Mid-Senior level,Full-time,Analyst,"Technology, Information and Internet",2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
2,3720365801,(Junior) Data Engineer (m/f/d),Sandbox Interactive,"Berlin, Berlin, Germany",1 day ago,YOUR GAMEBuild and maintain the current data i...,Mid-Senior level,Full-time,Information Technology,Computer Games,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
3,3672508925,Financial Data Analyst (m/f/d),Ultramarin,"Berlin, Berlin, Germany",2 months ago,About The PositionWe are looking for a top-tie...,Mid-Senior level,Full-time,Information Technology,Financial Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
4,3695466781,Business Intelligence Analyst (f/m/d),Les Lunes,"Berlin, Berlin, Germany",4 weeks ago,Your missionAs a Business Intelligence Analyst...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",Retail Apparel and Fashion,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
5,3585738180,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,Orange Quarter are working with an exciting st...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
6,3585734564,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,Orange Quarter are currently looking for a Dat...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
7,3704916810,BI Analyst (f/m/d),Enter,"Berlin, Berlin, Germany",3 weeks ago,Build the future with Enter.Our mission is to ...,Mid-Senior level,Full-time,Information Technology,Architecture and Planning,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
8,3585734595,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,We are currently working with an emerging SaaS...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
9,3698396795,Senior Data Analyst,Klim,"Berlin, Berlin, Germany",1 month ago,"We are Klim, a Berlin-based clean-tech company...",Mid-Senior level,Full-time,Other,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en


In [248]:


tqdm.pandas(desc="Processing")
nlp_df['cp1_title'] = nlp_df["title"].progress_apply(myNLP.clean_up)

Processing: 100%|██████████| 22079/22079 [00:00<00:00, 152467.04it/s]


remove text in parentheses

In [249]:
pattern_to_extract = r'\((.*?)\)'

# Extract text within parentheses and put it in a new column
nlp_df['text_in_parentheses'] = nlp_df['cp1_title'].progress_apply(lambda x: re.findall(pattern_to_extract, x))

# Convert lists with only one element to that element, and keep lists for multiple elements
nlp_df['text_in_parentheses'] = nlp_df['text_in_parentheses'].progress_apply(lambda x: x[0] if len(x) == 1 else x)

# Remove the text within parentheses 
nlp_df['cp1_title'] = nlp_df['cp1_title'].progress_apply(lambda x: re.sub(pattern_to_extract, '', x).strip())


Processing: 100%|██████████| 22079/22079 [00:00<00:00, 655015.12it/s]


### Tokenize, lemmatize, remove stopwords

In [250]:
nlp_df['cp2_title'] = nlp_df['cp1_title'].progress_apply(myNLP.tokenize)
nlp_df['cp2_title'] = nlp_df['cp2_title'].progress_apply(myNLP.stem_and_lemmatize)
nlp_df['cp2_title'] = nlp_df['cp2_title'].progress_apply(myNLP.remove_stopwords)
nlp_df['cp2_title'] = nlp_df['cp2_title'].progress_apply(myNLP.re_blob) 


Processing: 100%|██████████| 22079/22079 [00:00<00:00, 930237.14it/s]


#### clean dataframe based on several rounds of inspecting word clouds 



In [251]:
nlp_df_clean = pd.DataFrame(nlp_df) # load data

nlp_df_clean.to_csv("test_nlp_dataframe.csv", encoding='utf-8', index=False)

In [252]:
nlp_df_clean = nlp_df_clean.progress_apply(myNLP.extract_level, axis=1)

Processing: 100%|██████████| 22079/22079 [00:09<00:00, 2361.37it/s]


In [253]:
nlp_df_clean['job_level'].value_counts(dropna=False)

job_level
NaN              12417
manager_level     3217
senior_level      3184
consultant        1873
student_level     1043
junior_level       345
Name: count, dtype: int64

In [254]:
nlp_df_clean = nlp_df_clean[~nlp_df_clean['job_level'].isin(['senior_level', 'student_level', 'manager_level'])]
nlp_df_clean['job_level'].value_counts(dropna=False)

job_level
NaN             12417
consultant       1873
junior_level      345
Name: count, dtype: int64

In [259]:
nlp_df_clean['cp2_title'] = nlp_df_clean['cp2_title'].progress_apply(myNLP.combine_keywords)

Processing: 100%|██████████| 14635/14635 [00:00<00:00, 20960.58it/s]


tokenize again after combining the keywords

In [256]:
# nlp_df_clean['cp2_title'] = nlp_df_clean['cp2_title'].progress_apply(myNLP.tokenize)

extract new job title with combined keywords

In [260]:
nlp_df_clean = nlp_df_clean.progress_apply(myNLP.extract_keywords, axis=1)

Processing: 100%|██████████| 14635/14635 [00:05<00:00, 2578.05it/s]


In [264]:
nlp_df_clean['new_job_title'].

0                      NaN
1           [data_analyst]
2          [data_engineer]
3           [data_analyst]
4                      NaN
               ...        
22070     [sap_specialist]
22072      [test_engineer]
22074    [devops_engineer]
22075                  NaN
22078                  NaN
Name: new_job_title, Length: 14635, dtype: object

In [ ]:
nlp_df_clean['cp2_title'] = nlp_df_clean['cp2_title'].progress_apply(myNLP.drop_words)
myNLP.word_cloud(nlp_df_clean, 'cp2_title')

split dataframe

In [ ]:
print(nlp_df_clean.shape)
nlp_df_split = nlp_df_clean[nlp_df_clean['new_job_title'].isna()]

print(nlp_df_split.shape)

all_text = nlp_df_split['cp2_title']

bag = " ".join(all_text) # combining all job title 

#### word cloud 

In [ ]:
myNLP.word_cloud(nlp_df_split, 'cp2_title')

In [ ]:
from keybert import KeyBERT


Yake keyword extractor

In [ ]:
import yake

kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(bag)

In [ ]:
for kw in keywords:
    print(kw)

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

feature extraction:TF-IDF (Term Frequency-Inverse Document Frequency) 

In [ ]:
## Lowercasing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def tokenize_stemm_vectorize(df):
## Tokenization and Removing Stop Words
    stop_words = set(stopwords.words('english'))
    df['cp2_title'] = df['cp2_title'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))

    ## Stemming
    stemmer = PorterStemmer()
    df['cp2_title'] = df['cp2_title'].apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x)]))

    # Text Vectorization
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['cp2_title'])
    
    return X

X = tokenize_stemm_vectorize(nlp_df_split)



Model: KMeans

In [ ]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Initialize variables
silhouette_scores = []
inertia = []
# Loop through different numbers of clusters
for i in range(10, 30):  # Start from 2 as silhouette_score needs at least 2 clusters
    kmeans = KMeans(n_clusters=i, init='k-means++')
    kmeans.fit(X)
    cluster_labels = kmeans.labels_
    
    inertia.append(kmeans.inertia_)
    
    # Calculate the silhouette score and append to the list
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    # print(f"For n_clusters = {i}, the silhouette score is {silhouette_avg}")


In [ ]:

# Plotting the silhouette scores
plt.plot(range(10, 30), silhouette_scores)
plt.title('Silhouette Method')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()



plt.plot(range(10, 30), inertia)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
# Step 3: Clustering
kmeans = KMeans(n_clusters=26)  # Assuming 5 clusters
kmeans.fit(X)
nlp_df_split['cluster_kmeans'] = kmeans.labels_


In [ ]:
kmean_groups = nlp_df_split[['id', 'cp2_title', 'cluster_kmeans']]

with open("webscraper/model_data/kmean_groups_v2.csv", "wb") as file:
    kmean_groups.to_csv(file)

In [ ]:
kmean_model1 = {"dataframe":nlp_df_split,
                "X":X,
                'Prediction':kmean_groups}

<!-- Model: naive_bayes / MultinomialNB -->

In [ ]:
# from sklearn.naive_bayes import MultinomialNB

# clf = MultinomialNB()
# clf.fit(X)
# nlp_df_split['cluster_naive_bayes'] = clf.predict(X)



manually reserach: filter specific words 

In [ ]:
myNLP.word_cloud(nlp_df_split, 'cp2_title')

In [ ]:
filtered_df = nlp_df_split[nlp_df_split['cp2_title'].str.contains("software", case=False)]



In [ ]:
filtered_df[['cp2_title', 'cp1_title','title']]

In [ ]:
filtered_df['cp1_title'] = filtered_df['cp1_title'].progress_apply(myNLP.combine_keywords)

In [ ]:
filtered_df = filtered_df.progress_apply(myNLP.extract_keywords, axis=1)

In [ ]:
filtered_df[['cp2_title', 'cp1_title','title', 'new_job_title']]

In [ ]:
# keywords = ['data_analyst', 'ml_engineer', 'it_systemadmin', 'software_developer','scientist_','research_scientist', 'data_architect','java_software_engineer','data_engineer', 'database_datawarehouse', 'business_analyst', 'product_analyst', 
#                 'business_intelligence_analyst', 'full_stack', 'database_administrator', 'fp&a_analyst', 'product_analyst'
#                 'programm_manager', 'data_analytics', 'big_data_engineer/specialist', 'sap_specialist', 'devops_engineer', 'backend_developer', 'data_management'
#                'software_engineer', 'analysis_engineer', 'data_scientist', 'machine_learning_engineer', 'ai_engineer', 'controlling_', 'it_systemadmin'
#                'cloud_engineer', 'deep_learning', 'reporting_analyst', 'test_engineer', 'system_engineer', 'specialist', 'sap_specialist']


# for key in keywords:
    